In [ ]:
import os
import glob
import librosa
import numpy as np
from tqdm import tqdm # Thêm thư viện này để xem thanh tiến trình (progress bar)

In [4]:
# --- 0. Định nghĩa các hằng số và thư mục ---
RAW_DATA_DIR = "../data/raw/"
PROCESSED_DATA_DIR = "../data/processed/"

# Tạo thư mục processed nếu chưa có
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

# Ánh xạ nhãn từ tên file
emotion_map = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# --- 1. Hàm trích xuất đặc trưng cho MỘT file ---
# Chúng ta sẽ trích xuất 3 loại đặc trưng theo yêu cầu đề bài
def extract_features(file_path):
    try:
        # 1. Tải file âm thanh, chuẩn hóa tần số lấy mẫu về 22050 Hz
        y, sr = librosa.load(file_path, sr=22050)
        
        # 2. Trích xuất MFCCs (20 hệ số)
        # .T để chuyển vị (transpose) rồi lấy mean theo trục 0 (theo thời gian)
        mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20).T, axis=0)
        
        # 3. Trích xuất Energy (Sử dụng Root Mean Square Energy)
        # (Đây là một trong các yêu cầu "energy" của đề bài)
        rms = np.mean(librosa.feature.rms(y=y).T, axis=0)
        
        # 4. (Tùy chọn, nhưng rất tốt) Trích xuất Zero-Crossing Rate
        zcr = np.mean(librosa.feature.zero_crossing_rate(y=y).T, axis=0)
        
        # 5. Kết hợp tất cả lại thành 1 vector duy nhất
        # (20 MFCCs + 1 RMS + 1 ZCR = 22 đặc trưng)
        features = np.hstack((mfccs, rms, zcr))
        
        # 6. Lấy nhãn (label) từ tên file
        filename = os.path.basename(file_path)
        label = emotion_map[filename.split('-')[2]]
        
        return features, label
    
    except Exception as e:
        print(f"Lỗi khi xử lý file {file_path}: {e}")
        return None, None

# --- 2. Vòng lặp xử lý tất cả file ---
all_features = []
all_labels = []

print("Bắt đầu trích xuất đặc trưng...")

# Lấy tất cả đường dẫn file .wav trong data/raw/
file_paths = glob.glob(os.path.join(RAW_DATA_DIR, "Actor_*", "*.wav"))

# Sử dụng tqdm để xem thanh tiến trình
for file_path in tqdm(file_paths):
    features, label = extract_features(file_path)
    
    if features is not None:
        all_features.append(features)
        all_labels.append(label)

print(f"\nTrích xuất hoàn tất. Xử lý được {len(all_features)} / {len(file_paths)} file.")

# --- 3. Lưu kết quả ra file ---
# Chuyển list thành mảng NumPy
X = np.array(all_features)
y = np.array(all_labels)

# In ra shape để kiểm tra
# X.shape sẽ là (1440, 22) - 1440 file, 22 đặc trưng
# y.shape sẽ là (1440,) - 1440 nhãn
print(f"Mảng đặc trưng (X) có hình dạng: {X.shape}")
print(f"Mảng nhãn (y) có hình dạng: {y.shape}")

# Lưu vào thư mục processed
np.save(os.path.join(PROCESSED_DATA_DIR, 'features.npy'), X)
np.save(os.path.join(PROCESSED_DATA_DIR, 'labels.npy'), y)

print("Đã lưu features.npy và labels.npy vào thư mục 'data/processed/'")

Bắt đầu trích xuất đặc trưng...


100%|██████████| 1440/1440 [00:35<00:00, 40.62it/s]


Trích xuất hoàn tất. Xử lý được 1440 / 1440 file.
Mảng đặc trưng (X) có hình dạng: (1440, 22)
Mảng nhãn (y) có hình dạng: (1440,)
Đã lưu features.npy và labels.npy vào thư mục 'data/processed/'
